### 🛠️ Setup

Import packages

In [1]:
import os, subprocess, sys, deepreload, pandas as pd, json
from sm.prelude import M

# WD_DATE = "20230619"
# WD_DATE = "20211213"
WD_DATE = "20230201"
WP_DATE = "20230620"
DBP_DATE = "20221201"

os.environ["WD_DIR"] = os.path.expanduser(f"~/kgdata/wikidata/{WD_DATE}")
os.environ["WP_DIR"] = os.path.expanduser(f"~/kgdata/wikipedia/{WP_DATE}")
os.environ["DBP_DIR"] = os.path.expanduser(f"~/kgdata/dbpedia/{DBP_DATE}")
os.environ["spark_master"] = "spark://ckg05.isi.edu:17077"

from kgdata.config import init_dbdir_from_env, WikidataDirCfg
from kgdata.dataset import import_dataset

init_dbdir_from_env()

2024-07-21 05:31:13.375 | INFO     | kgdata.config:init_dbdir_from_env:26 - Wikidata directory: /nas/home/binhvu/kgdata/wikidata/20230201
2024-07-21 05:31:13.377 | INFO     | kgdata.config:init_dbdir_from_env:33 - DBpedia directory: /nas/home/binhvu/kgdata/dbpedia/20221201
2024-07-21 05:31:13.379 | INFO     | kgdata.config:init_dbdir_from_env:42 - Wikipedia directory: /nas/home/binhvu/kgdata/wikipedia/20230620


### Main

In [2]:
import orjson
from kgdata.spark.common import get_spark_context, close_spark_context

In [21]:
close_spark_context()

#### Develop Wikidata pages-articles parser

In [3]:
# from kgdata.wikidata.datasets.page_article_dump import *
from kgdata.wikidata.datasets.triple_truthy_dump import *
from kgdata.wikidata.datasets.triple_truthy_dump_derivatives import *
from kgdata.wikidata.models.wdentity import WDEntity

In [4]:
ds = triple_truthy_dump()

In [11]:
ds.get_rdd().map(lambda x: x.id).saveAsTextFile("/var/tmp/testid")

In [16]:
from urllib.parse import urlparse

def test(id):
    if not id.startswith("http"):
        return None
    return urlparse(id).netloc

In [17]:
netloc = get_spark_context().textFile("/var/tmp/testid").map(test).filter(lambda x: x is not None).distinct().collect()

In [20]:
resp = ds.get_rdd().flatMap(lambda x: x.props.keys()).distinct().collect()

In [ ]:
resp = ds.get_rdd().filter(lambda x: x.id.find("wikipedia") != -1).take(1)

In [6]:
resp = ds.get_rdd().map(get_site_links).filter(lambda x: x is not None).take(1)

In [10]:
resp

[]

In [21]:
resp = ds.get_rdd().filter(lambda x: x.id == 'Ndd357270c5e240d9b8fee48637e0f3c0').take(1)

In [18]:
!python -m noti slack done

In [36]:
ignore_props = {'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
 'http://www.w3.org/2002/07/owl#onProperty',
 'http://www.w3.org/2002/07/owl#someValuesFrom'}
oldds = Dataset("/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240320/013_triple_truthy_dump/final-2/*.zst", deserialize=RDFResource.deser)

def filter_bnode(x):
    if x.id.startswith("http"):
        return False
    props = {k: p for k, p in x.props.items() if k not in ignore_props}
    return len(props) > 0
    
resp = oldds.get_rdd().filter(filter_bnode).take(100)

In [40]:
len(resp)

0

In [17]:
resp = (
    ds.get_rdd()
    .filter(lambda x: x.id.startswith("http://www.wikidata.org/entity/"))
    .filter(lambda x: x.id == 'http://www.wikidata.org/entity/Q62217978')
    .take(1)
)

2024-07-17 22:47:42.669 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: [('spark.master', 'spark://ckg05.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g')]


In [35]:
{k for x in resp for k in x.props.keys()}

{'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
 'http://www.w3.org/2002/07/owl#onProperty',
 'http://www.w3.org/2002/07/owl#someValuesFrom'}

In [23]:
resp[0].to_dict()

IndexError: list index out of range

In [37]:
!python -m noti slack done

get list of properties

In [7]:
from kgdata.wikidata.datasets.entities_from_truthy_dump import *

In [6]:
import rdflib

In [7]:
value = rdflib.term.Literal('Point(-93.69681 40.38078)', datatype=rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#wktLiteral'))

In [11]:
import re

m = re.match(r"Point\((?P<long>-?[0-9\.]+) (?P<lat>-?[0-9\.]+)\)", str(value))
m.group("long")

'-93.69681'

In [9]:
resp = ds.get_rdd().filter(lambda x: x.id.startswith("http://www.wikidata.org/entity/")).filter(lambda x: rdf_type not in x.props).take(10)

In [12]:
resp[1]

RDFResource(id='http://www.wikidata.org/entity/Q9934455', props={'http://www.w3.org/2002/07/owl#sameAs': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q6805848')]})

In [10]:
!python -m noti slack done

In [4]:
entities_from_truthy_dump()

2024-07-10 06:05:18.714 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: [('spark.master', 'spark://ckg05.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g')]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/10 06:05:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


read dataset:   0%|          | 0/64 [00:00<?, ?it/s]

24/07/10 06:06:30 WARN TaskSetManager: Lost task 48.0 in stage 1.0 (TID 325) (128.9.35.39 executor 0): org.apache.spark.SparkException: Task failed while writing rows
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:163)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:161)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.base/java.util.concurrent.Th

Py4JJavaError: An error occurred while calling o49.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:407)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope

he.spark.internal.io.SparkHadoopWriter$.$anonfun$executeTask$1(SparkHadoopWriter.scala:136)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1397)
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:135)
	... 12 more

Driver stacktrace:)
24/07/10 06:07:18 WARN TaskSetManager: Lost task 174.3 in stage 1.0 (TID 519) (128.9.35.39 executor 0): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 40 in stage 1.0 failed 4 times, most recent failure: Lost task 40.3 in stage 1.0 (TID 496) (128.9.35.39 executor 0): org.apache.spark.SparkException: Task failed while writing rows
	at org.apache.spark.internal.io.SparkHadoopWriter$.executeTask(SparkHadoopWriter.scala:163)
	at org.apache.spark.internal.io.SparkHadoopWriter$.$anonfun$write$1(SparkHadoopWriter.scala:88)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:161)
	

In [15]:
lst_props = entities_from_truthy_dump().get_list()

read dataset:   0%|          | 0/64 [00:00<?, ?it/s]

In [18]:
wikibase_propertyType = "http://wikiba.se/ontology#propertyType"

In [21]:
lst_props[0].id

'http://www.wikidata.org/entity/P11121'

In [22]:
for x in lst_props:
    assert len(x.props[wikibase_propertyType]) == 1

In [20]:
{y for x in lst_props for y in x.props[wikibase_propertyType]}

{rdflib.term.URIRef('http://wikiba.se/ontology#CommonsMedia'),
 rdflib.term.URIRef('http://wikiba.se/ontology#ExternalId'),
 rdflib.term.URIRef('http://wikiba.se/ontology#GeoShape'),
 rdflib.term.URIRef('http://wikiba.se/ontology#GlobeCoordinate'),
 rdflib.term.URIRef('http://wikiba.se/ontology#Math'),
 rdflib.term.URIRef('http://wikiba.se/ontology#Monolingualtext'),
 rdflib.term.URIRef('http://wikiba.se/ontology#MusicalNotation'),
 rdflib.term.URIRef('http://wikiba.se/ontology#Quantity'),
 rdflib.term.URIRef('http://wikiba.se/ontology#String'),
 rdflib.term.URIRef('http://wikiba.se/ontology#TabularData'),
 rdflib.term.URIRef('http://wikiba.se/ontology#Time'),
 rdflib.term.URIRef('http://wikiba.se/ontology#Url'),
 rdflib.term.URIRef('http://wikiba.se/ontology#WikibaseForm'),
 rdflib.term.URIRef('http://wikiba.se/ontology#WikibaseItem'),
 rdflib.term.URIRef('http://wikiba.se/ontology#WikibaseLexeme'),
 rdflib.term.URIRef('http://wikiba.se/ontology#WikibaseProperty'),
 rdflib.term.URIRef

In [ ]:
def is_property(r):
    if r.props.get(rdf_type)
lst_props = ds.get_rdd().filter(is_property).collect()

get K entities

In [8]:
def is_entity(x):
    return x.id.startswith("http://www.wikidata.org/entity/")

lst_ents = ds.get_rdd().filter(is_entity).take(5000)

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x7fa74fe23600>
Traceback (most recent call last):
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/isodate/isodatetime.py", line 55, in parse_datetime
    tmpdate = parse_date(datestring)
              ^^^^^^^^^^^^^^^^^^^^^^
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0317-07-15'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#

In [14]:
for ent in lst_ents[2:]:
    print(ent.id, ent.props.get(skos_prefLabel, []))
    break

http://www.wikidata.org/entity/Q12574348 [rdflib.term.Literal('Қантентек Жаманғұтов', lang='kk')]


In [11]:
print(json.dumps(lst_ents[0].to_dict(), indent=2))

{
  "id": "http://www.wikidata.org/entity/Q37815329",
  "props": {
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#type": [
      {
        "type": "URIRef",
        "value": "http://wikiba.se/ontology#Item"
      }
    ],
    "http://www.wikidata.org/prop/direct/P762": [
      {
        "type": "Literal",
        "value": "48135/7-7449",
        "datatype": null,
        "language": null
      }
    ],
    "http://www.wikidata.org/prop/direct/P17": [
      {
        "type": "URIRef",
        "value": "http://www.wikidata.org/entity/Q213"
      }
    ],
    "http://www.wikidata.org/prop/direct/P1435": [
      {
        "type": "URIRef",
        "value": "http://www.wikidata.org/entity/Q385405"
      }
    ],
    "http://www.wikidata.org/prop/direct/P18": [
      {
        "type": "URIRef",
        "value": "http://commons.wikimedia.org/wiki/Special:FilePath/Pansk%C3%A1%2016%2C%20Brno%20%282%29.JPG"
      }
    ],
    "http://www.wikidata.org/prop/direct/P131": [
      {
        "type":

get id prefix

In [23]:
def is_entity(x):
    return x.id.startswith("http://www.wikidata.org/entity/")

ent_prefixes = ds.get_rdd().filter(is_entity).map(lambda x: x.id[len("http://www.wikidata.org/entity/")]).distinct().collect()

In [24]:
ent_prefixes

['P', 'Q']

In [25]:
!python -m noti slack done

In [74]:
deepreload.reload("kgdata.wikidata.datasets.entity_triple_truthy_dump")
from kgdata.wikidata.datasets.entity_triple_truthy_dump import *

In [67]:
res = ds.take(10)

read dataset:   0%|          | 0/842 [00:00<?, ?it/s]

In [66]:
res[0]

RDFResource(id='https://www.wikidata.org/wiki/Special:EntityData/Q119372500', props={'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://schema.org/Dataset')], 'http://schema.org/about': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q119372500')], 'http://schema.org/version': [rdflib.term.Literal('2100932230', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))], 'http://schema.org/dateModified': [rdflib.term.Literal('2024-03-13T08:23:12+00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))]})

In [70]:
res[4]

RDFResource(id='http://www.wikidata.org/entity/Q73972426', props={'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://wikiba.se/ontology#Item')], 'http://www.wikidata.org/prop/direct/P1433': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q15752473')], 'http://www.wikidata.org/prop/direct/P1476': [rdflib.term.Literal('The problems of homoplastic grafting, as applied to the cornea', lang='en')], 'http://www.wikidata.org/prop/direct/P2093': [rdflib.term.Literal('S DUKE-ELDER')], 'http://www.wikidata.org/prop/direct/P304': [rdflib.term.Literal('187-212')], 'http://www.wikidata.org/prop/direct/P31': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q13442814')], 'http://www.wikidata.org/prop/direct/P433': [rdflib.term.Literal('3')], 'http://www.wikidata.org/prop/direct/P478': [rdflib.term.Literal('1')], 'http://www.wikidata.org/prop/direct/P577': [rdflib.term.Literal('1956-03-01T00:00:00+00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#

In [25]:
def to_entity(resource: RDFResource, dump_lang: str) -> Optional[WDEntity]:
    id = resource.id
    if id.startswith("https://www.wikidata.org/wiki/Special:EntityData/"):
        # skip this Special:EntityData/XXX, there is a real record XXX
        return None
        
    if id.startswith("http://www.wikidata.org/entity/"):
        # we found an entity
        label = extract 
        return 1
    
    if id.startswith("http://www.wikidata.org/prop/"):
        return False
    if not id.startswith("http"):
        assert len(id) == len("N2efd4b1ea98e4135b0f8b0bec9e7c471"), id
        return False
    return True
    
    # label = extract_entity_label(resource, dump_lang)
    # default_lang = label.lang

    # description = as_multilingual(
    #     assert_all_literal(resource.props.get(rdfs_comment, []))
    # )
    # if len(description.lang2value) > 1:
    #     if default_lang in description.lang2value:
    #         description.lang = default_lang
    #     else:
    #         # to handle case where there is no description in default language
    #         description.lang = default_lang
    #         description.lang2value[default_lang] = ""

    # return Entity(
    #     id=resource.id,
    #     label=label,
    #     description=description,
    #     aliases=MultiLingualStringList({default_lang: []}, default_lang),
    #     props={
    #         pid: [to_statement(s) for s in lst] for pid, lst in resource.props.items()
    #     },
    # )

In [ ]:
!python -m noti slack done

In [34]:
def nonwiki_props(res: RDFResource):
    out = []
    for prop in res.props.keys():
        if not prop.startswith("http://www.wikidata.org/prop/"):
            out.append((prop, 1))
    return out

In [35]:
prop2count = ds.get_rdd().flatMap(nonwiki_props).reduceByKey(lambda a, b: a + b).collect()

In [119]:
s  = "http://www.wikidata.org/entity/Q012983"
s[len("http://www.wikidata.org/entity/")]

'Q'

In [120]:
def type2prop(resource: RDFResource):
    id = resource.id
    out = []
    if id.startswith("https://www.wikidata.org/wiki/Special:EntityData/"):
        # skip this Special:EntityData/XXX, there is a real record XXX
        return out
    if id == 'http://wikiba.se/ontology#Dump':
        return out
        
    if id.startswith("http://www.wikidata.org/entity/"):
        # we found an entity
        for prop in resource.props.keys():
            if not prop.startswith("http://www.wikidata.org/prop/"):
                out.append(('entity:' + resource.id[len("http://www.wikidata.org/entity/")] + ":" + prop, 1))
        return out
    
    if id.startswith("http://www.wikidata.org/prop/"):
        for prop in resource.props.keys():
            if not prop.startswith("http://www.wikidata.org/prop/"):
                out.append(('prop:' + resource.id[len("http://www.wikidata.org/prop/")] + ":" + prop, 1))
        return out
        
    if not id.startswith("http"):
        assert len(id) == len("N2efd4b1ea98e4135b0f8b0bec9e7c471"), id
        for prop in resource.props.keys():
            if not prop.startswith("http://www.wikidata.org/prop/"):
                out.append(('bnode:' + prop, 1))
        return out

    assert False

In [105]:
from rdflib.namespace import split_uri

In [107]:
split_uri("http://www.w3.org/2000/01/rdf-schema/label")

('http://www.w3.org/2000/01/rdf-schema/', 'label')

In [130]:
def nonwiki_props(res: RDFResource):
    out = []
    for prop in res.props.keys():
        if prop in {rdf_type, rdfs_label, schema_name, skos_prefLabel, skos_altLabel, schema_description}:
            continue
        out.append((split_uri(prop)[0], 1))
    return out

In [131]:
ent_prop2count = (
    ds.get_rdd()
    .filter(lambda r: r.id.startswith("http://www.wikidata.org/entity/Q"))
    .flatMap(nonwiki_props)
    .reduceByKey(lambda a, b: a + b).collect()
)

In [142]:
wikibase_property = URIRef('http://wikiba.se/ontology#Property')
def is_property(res):
    return wikibase_property in res.props.get(rdf_type, [])

In [143]:
props = ds.get_rdd().filter(is_property).collect()

In [155]:
all_types = ds.get_rdd() \
    .flatMap(lambda r: [
        x for x in r.props.get(rdf_type, []) 
        if not x.startswith("http://www.wikidata.org/prop/novalue/")
    ]).distinct().take(500)

In [156]:
!python -m noti slack done

In [133]:
resp = ds.get_rdd().filter(lambda x: x.id.startswith("http://www.wikidata.org/entity/")).take(100)

In [149]:
to_dict(props[0])

{'id': 'http://www.wikidata.org/entity/P4718',
 'props': {'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [{'type': 'URIRef',
    'value': 'http://wikiba.se/ontology#Property'}],
  'http://www.wikidata.org/prop/direct/P31': [{'type': 'URIRef',
    'value': 'http://www.wikidata.org/entity/Q107737764'}],
  'http://www.wikidata.org/prop/direct/P3254': [{'type': 'URIRef',
    'value': 'https://www.wikidata.org/wiki/Wikidata:Property_proposal/Basketball-Reference.com_NBA_coach_ID'}],
  'http://www.wikidata.org/prop/direct/P1630': [{'type': 'Literal',
    'value': 'https://www.basketball-reference.com/coaches/$1.html',
    'datatype': None,
    'language': None}],
  'http://www.wikidata.org/prop/direct/P1896': [{'type': 'URIRef',
    'value': 'https://www.basketball-reference.com/coaches/'}],
  'http://www.wikidata.org/prop/direct/P1855': [{'type': 'URIRef',
    'value': 'http://www.wikidata.org/entity/Q988438'},
   {'type': 'URIRef', 'value': 'http://www.wikidata.org/entity/Q39141916'},


In [145]:
len(props)

11731

In [153]:
resp[1]

RDFResource(id='http://www.wikidata.org/entity/Q12358696', props={'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://wikiba.se/ontology#Item')], 'http://www.wikidata.org/prop/direct/P31': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q5')], 'http://www.wikidata.org/prop/direct/P21': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q6581097')], 'http://www.wikidata.org/prop/direct/P735': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q17520844')], 'http://www.wikidata.org/prop/direct/P569': [rdflib.term.Literal('1932-01-01T00:00:00+00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))], 'http://www.wikidata.org/prop/direct/P570': [rdflib.term.Literal('2012-09-05T00:00:00+00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))], 'http://www.wikidata.org/prop/direct/P27': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q191')], 'http://www.w3.org/2000/01/rdf-schema#label': [rdflib.term.Lite

In [140]:
resp[0].props['http://www.wikidata.org/prop/direct-normalized/P762'][0]

KeyError: 'http://www.wikidata.org/prop/direct-normalized/P762'

In [121]:
prop2count = ds.get_rdd().flatMap(type2prop).reduceByKey(lambda a, b: a + b).collect()

In [132]:
!python -m noti slack done

In [127]:
[x[0] for x in prop2count if x[0].startswith("entity:")]

['entity:Q:http://www.w3.org/2002/07/owl#sameAs',
 'entity:P:http://www.w3.org/2004/02/skos/core#prefLabel',
 'entity:Q:http://www.w3.org/2000/01/rdf-schema#label',
 'entity:P:http://wikiba.se/ontology#referenceValue',
 'entity:P:http://wikiba.se/ontology#novalue',
 'entity:P:http://wikiba.se/ontology#qualifierValueNormalized',
 'entity:P:http://wikiba.se/ontology#qualifier',
 'entity:P:http://www.w3.org/2004/02/skos/core#altLabel',
 'entity:P:http://wikiba.se/ontology#statementProperty',
 'entity:P:http://wikiba.se/ontology#referenceValueNormalized',
 'entity:P:http://wikiba.se/ontology#directClaim',
 'entity:Q:http://schema.org/description',
 'entity:P:http://schema.org/description',
 'entity:P:http://wikiba.se/ontology#reference',
 'entity:P:http://wikiba.se/ontology#propertyType',
 'entity:P:http://wikiba.se/ontology#statementValueNormalized',
 'entity:P:http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
 'entity:P:http://schema.org/name',
 'entity:P:http://wikiba.se/ontology#qualif

In [110]:
!python -m noti slack done

In [126]:
ent_prop2count

[('http://www.wikidata.org/prop/direct/', 1010648296),
 ('http://schema.org/', 208555833),
 ('http://wikiba.se/ontology#', 154606),
 ('http://www.wikidata.org/prop/direct-normalized/', 102923517),
 ('http://www.w3.org/2004/02/skos/core#', 121256902),
 ('http://www.w3.org/1999/02/22-rdf-syntax-ns#', 107746649),
 ('http://www.w3.org/2000/01/rdf-schema#', 107554477),
 ('http://www.w3.org/2002/07/owl#', 4282604)]

In [82]:
samples[0].props

{'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://wikiba.se/ontology#Item')],
 'http://www.wikidata.org/prop/direct/P762': [rdflib.term.Literal('48135/7-7449')],
 'http://www.wikidata.org/prop/direct/P17': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q213')],
 'http://www.wikidata.org/prop/direct/P1435': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q385405')],
 'http://www.wikidata.org/prop/direct/P18': [rdflib.term.URIRef('http://commons.wikimedia.org/wiki/Special:FilePath/Pansk%C3%A1%2016%2C%20Brno%20%282%29.JPG')],
 'http://www.wikidata.org/prop/direct/P131': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q12039338')],
 'http://www.wikidata.org/prop/direct/P625': [rdflib.term.Literal('Point(16.607861111111 49.193611111111)', datatype=rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#wktLiteral'))],
 'http://www.wikidata.org/prop/direct/P31': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q1078374')],
 'http://www.wikidata.

In [37]:
prop2count

[('http://www.w3.org/2000/01/rdf-schema#label', 107554477),
 ('http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 219930509),
 ('http://schema.org/softwareVersion', 1),
 ('http://wikiba.se/ontology#qualifierValueNormalized', 9324),
 ('http://wikiba.se/ontology#propertyType', 11731),
 ('http://wikiba.se/ontology#directClaimNormalized', 9324),
 ('http://www.w3.org/2002/07/owl#sameAs', 4282604),
 ('http://wikiba.se/ontology#reference', 11731),
 ('http://www.w3.org/2002/07/owl#someValuesFrom', 11731),
 ('http://schema.org/about', 112029253),
 ('http://schema.org/dateModified', 112029254),
 ('http://wikiba.se/ontology#statementProperty', 11731),
 ('http://wikiba.se/ontology#qualifierValue', 11731),
 ('http://schema.org/name', 107554477),
 ('http://www.w3.org/2002/07/owl#imports', 1),
 ('http://schema.org/version', 112029253),
 ('http://wikiba.se/ontology#qualifier', 11731),
 ('http://www.w3.org/2002/07/owl#onProperty', 11731),
 ('http://www.w3.org/2004/02/skos/core#prefLabel', 107554477),
 (

In [38]:
w

RDFResource(id='http://www.wikidata.org/entity/Q37815329', props={'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://wikiba.se/ontology#Item')], 'http://www.wikidata.org/prop/direct/P762': [rdflib.term.Literal('48135/7-7449')], 'http://www.wikidata.org/prop/direct/P17': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q213')], 'http://www.wikidata.org/prop/direct/P1435': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q385405')], 'http://www.wikidata.org/prop/direct/P18': [rdflib.term.URIRef('http://commons.wikimedia.org/wiki/Special:FilePath/Pansk%C3%A1%2016%2C%20Brno%20%282%29.JPG')], 'http://www.wikidata.org/prop/direct/P131': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q12039338')], 'http://www.wikidata.org/prop/direct/P625': [rdflib.term.Literal('Point(16.607861111111 49.193611111111)', datatype=rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#wktLiteral'))], 'http://www.wikidata.org/prop/direct/P31': [rdflib.term.URIRef('http://w

In [42]:
!python -m noti slack done

In [31]:
samples[0]

RDFResource(id='http://www.wikidata.org/entity/Q37815329', props={'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': [rdflib.term.URIRef('http://wikiba.se/ontology#Item')], 'http://www.wikidata.org/prop/direct/P762': [rdflib.term.Literal('48135/7-7449')], 'http://www.wikidata.org/prop/direct/P17': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q213')], 'http://www.wikidata.org/prop/direct/P1435': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q385405')], 'http://www.wikidata.org/prop/direct/P18': [rdflib.term.URIRef('http://commons.wikimedia.org/wiki/Special:FilePath/Pansk%C3%A1%2016%2C%20Brno%20%282%29.JPG')], 'http://www.wikidata.org/prop/direct/P131': [rdflib.term.URIRef('http://www.wikidata.org/entity/Q12039338')], 'http://www.wikidata.org/prop/direct/P625': [rdflib.term.Literal('Point(16.607861111111 49.193611111111)', datatype=rdflib.term.URIRef('http://www.opengis.net/ont/geosparql#wktLiteral'))], 'http://www.wikidata.org/prop/direct/P31': [rdflib.term.URIRef('http://w

In [7]:
ds.get_rdd().filter(lambda x: to_entity(x, "en")).count()

112207322

In [18]:
ds.get_rdd().filter(lambda x: to_entity(x, "en")).count()

178069

In [12]:
resp = ds.get_rdd().filter(lambda x: x.id == "http://www.wikidata.org/entity/Q29522960").take(1)

In [26]:
ids = ds.get_rdd().filter(lambda x: to_entity(x, "en")).take(100)

In [29]:
[x.id for x in ids]

['http://wikiba.se/ontology#Dump']

In [28]:
!python -m noti slack done

#### Parse Page Article Dump

In [5]:
rdd = get_spark_context().textFile(
    "/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/014_page_article_dump/raw-xml"
).map(orjson.loads)

In [21]:
ff = {"Q93565803", "Q93565810", "Q93565827", "Q93565832", "Q93565884", "Q93565766", "Q93565779", "Q93565785", "Q93565792", "Q93565795"}

def parse(x):
    el = etree.fromstring(x)
    title = el.find("{*}title").text
    (revision,) = el.findall("{*}revision")

    format = revision.find("{*}format").text
    text = revision.find("{*}text").text

    if format == "application/json":
        try:
            orjson.loads(text)
        except:
            assert text == "" or text is None
            return True
    return False
    # if title in ff:
    #     return True
    # return False

In [54]:
el = etree.fromstring(all_errordata[0])

In [60]:
el.findall("{*}revision")[0].find("{*}text").text is None

True

24/07/06 22:15:32 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/07/06 22:15:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [7]:
errordata = rdd.filter(parse).collect()

In [20]:
print(errordata[10])

IndexError: list index out of range

In [22]:
all_errordata = rdd.filter(parse).collect()

In [24]:
len(all_errordata)

535

In [43]:
print(all_errordata[0])

<page xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <title>Q93532819</title>
    <ns>0</ns>
    <id>92625985</id>
    <revision>
      <id>1702365186</id>
      <parentid>1511654627</parentid>
      <timestamp>2022-08-12T11:18:18Z</timestamp>
      <contributor>
        <username>Renamerr</username>
        <id>2883061</id>
      </contributor>
      <comment>/* wbsetdescription-add:1|uk */ &#1075;&#1072;&#1083;&#1072;&#1082;&#1090;&#1080;&#1082;&#1072;, [[:toollabs:quickstatements/#/batch/95057|batch #95057]]</comment>
      <model>wikibase-item</model>
      <format>application/json</format>
      <text bytes="21152"/>
      <sha1>h5rlc7fjediv4htxd38utd98deu4j8a</sha1>
    </revision>
  </page>
  


In [49]:
!python -m noti slack done

In [50]:

def missing_entities(articles: list):
    fixed = {}
    for article in tqdm(articles):
        el = etree.fromstring(article)
        title = el.find("{*}title").text

        (revision,) = el.findall("{*}revision")
        format = revision.find("{*}format").text

        assert format == "application/json"
        revision_id = revision.find("{*}id").text

        resp = requests.get(
            "https://www.wikidata.org/w/api.php",
            params={
                "action": "query",
                "format": "json",
                "formatversion": "2",
                "prop": "revisions",
                "revids": revision_id,
                "rvprop": "timestamp|comment|content",
                "rvslots": "main",
            },
        )
        assert resp.status_code == 200
        data = resp.json()
        (data_page,) = data["query"]["pages"]
        assert data_page["title"] == title, (data_page["title"], title)
        (data_revision,) = data_page["revisions"]
        data_revision["slots"]["main"]["content"]

        fixed[title] = data_revision["slots"]["main"]["content"]
    return fixed


In [46]:
from tqdm.auto import tqdm

In [51]:
fixed = missing_entities(all_errordata)

  0%|          | 0/535 [00:00<?, ?it/s]

In [52]:
with open("/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/014_page_article_dump/fixed-dumps.json", "w") as f:
    f.write(orjson.dumps(fixed).decode())

In [53]:
!python -m noti slack done

In [28]:
import requests

resp = requests.get("https://www.wikidata.org/w/api.php?action=query&format=json&formatversion=2&prop=revisions&revids=1702365186&rvprop=timestamp|comment|content&rvslots=main")

In [44]:
resp.json()['query']['pages']

[{'pageid': 92625985,
  'ns': 0,
  'title': 'Q93532819',
  'revisions': [{'timestamp': '2022-08-12T11:18:18Z',
    'slots': {'main': {'contentmodel': 'wikibase-item',
      'contentformat': 'application/json',
      'content': '{"type":"item","id":"Q93532819","labels":{"en":{"language":"en","value":"2MASX J21344437-4209281"},"nl":{"language":"nl","value":"2MASX J21344437-4209281"},"bn":{"language":"bn","value":"\\u09e8\\u09ae\\u09cd\\u09af\\u09be\\u09b8\\u098f\\u0995\\u09cd\\u09b8 \\u099c\\u09c7\\u09e8\\u09e7\\u09e9\\u09ea\\u09ea\\u09ea\\u09e9\\u09ed-\\u09ea\\u09e8\\u09e6\\u09ef\\u09e8\\u09ee\\u09e7"},"de":{"language":"de","value":"2MASX J21344437-4209281"},"la":{"language":"la","value":"2MASX J21344437-4209281"},"pt":{"language":"pt","value":"2MASX J21344437-4209281"},"pl":{"language":"pl","value":"2MASX J21344437-4209281"},"gl":{"language":"gl","value":"2MASX J21344437-4209281"},"io":{"language":"io","value":"2MASX J21344437-4209281"},"eu":{"language":"eu","value":"2MASX J21344437-42

In [26]:
!wget 

zsh:1: parse error near `&'


In [5]:
ds = page_article_dump()

In [5]:
get_spark_context().textFile(
    "/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/014_page_article_dump/raw"
).coalesce(512, shuffle=True).saveAsTextFile(
    "/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/051_abc",
    compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec",
)

In [5]:
get_spark_context().textFile(
    "/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/014_page_article_dump/raw"
).count()

2024-06-28 06:43:31.258 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: [('spark.master', 'spark://ckg04.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g')]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/28 06:43:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


112858294

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50270)
Traceback (most recent call last):
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/pys

In [9]:
ds.get_rdd().map(lambda x: x["title"]).distinct().count()

112858294

In [5]:
ds.get_rdd().map(lambda x: x["title"] == "Q30").distinct().count()

2024-07-06 04:31:26.703 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: [('spark.master', 'spark://ckg04.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g')]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/06 04:31:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2

In [6]:
q30 = ds.get_rdd().filter(lambda x: x["title"] == "Q30").collect()

2024-07-06 04:49:23.894 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: [('spark.master', 'spark://ckg04.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g')]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/06 04:49:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/06 04:49:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/06 04:49:45 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/07/06 04:50:00 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/07/06 04:50:15 WARN TaskSchedulerImpl: Initial job has not a

In [9]:
def bad_text(obj):
    if obj["revision"]["text"] is None:
        return False
    try:
        orjson.loads(obj["revision"]["text"])
        return False
    except:
        return True

In [11]:
ds.get_rdd().filter(lambda x: x['revision']['text'] is None).count()

535

In [10]:
ds.get_rdd().filter(lambda x: bad_text(x)).count()

0

In [12]:
records = ds.get_rdd().filter(lambda x: x['revision']['text'] is None).collect()

In [16]:
records[0]

{'id': '92656913',
 'ns': '0',
 'title': 'Q93565803',
 'revision': {'id': '1705846995',
  'parent_id': '1627014292',
  'timestamp': '2022-08-16T23:26:34Z',
  'model': 'wikibase-item',
  'format': 'application/json',
  'text': None}}

In [17]:
cfg = WikidataDirCfg.get_instance()

In [18]:
files = cfg.get_page_article_dump_files()

In [19]:
files[0]

PosixPath('/nas/home/binhvu/kgdata/wikidata/20240220/000_dumps/pages-articles/wikidatawiki-20240220-pages-articles-multistream6.xml-p5969005p6052571.bz2')

In [21]:
from kgdata.wikidata.datasets.page_article_dump import *

In [22]:
def parse_page_article_dump(infile: Path):
    it = iter_parse_page_article_dump(infile)

    # reparsing the header first to ensure the structure is right
    event, el = next(it)
    assert (
        event == "start"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}mediawiki"
    )

    event, el = next(it)
    assert (
        event == "start"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}siteinfo"
    )
    for event, el2 in it:
        if event == "end" and el2 is el:
            break
    assert el.find("{*}sitename").text == "Wikidata"
    print(etree.tostring(el))
    el.clear()

    

In [23]:
parse_page_article_dump(files[0])

b'<siteinfo xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n    <sitename>Wikidata</sitename>\n    <dbname>wikidatawiki</dbname>\n    <base>https://www.wikidata.org/wiki/Wikidata:Main_Page</base>\n    <generator>MediaWiki 1.42.0-wmf.18</generator>\n    <case>first-letter</case>\n    <namespaces>\n      <namespace key="-2" case="first-letter">Media</namespace>\n      <namespace key="-1" case="first-letter">Special</namespace>\n      <namespace key="0" case="first-letter"/>\n      <namespace key="1" case="first-letter">Talk</namespace>\n      <namespace key="2" case="first-letter">User</namespace>\n      <namespace key="3" case="first-letter">User talk</namespace>\n      <namespace key="4" case="first-letter">Wikidata</namespace>\n      <namespace key="5" case="first-letter">Wikidata talk</namespace>\n      <namespace key="6" case="first-letter">File</namespace>\n      <namespace key="7" case="first-letter">File talk</namespace>\n

In [15]:
pd.DataFrame([{'title': x['title']} for x in records])

,title
0,Q93565803
1,Q93565810
2,Q93565827
3,Q93565832
4,Q93565884
...,...
530,Q93565766
531,Q93565779
532,Q93565785
533,Q93565792


In [5]:
lst = (
    ds.get_rdd()
    # .filter(lambda x: not (x["title"][0] == "Q" and x["title"][1:].isdigit()))
    # .filter(lambda x: bad_text(x))
    .filter(lambda x: len(x["revision"]) != -1).take(10)
)

2024-06-28 07:45:29.019 | DEBUG    | kgdata.spark.common:get_spark_context:107 - Spark Options: [('spark.master', 'spark://ckg04.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g')]


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/28 07:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58684)
Traceback (most recent call last):
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/nas/home/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/nas/home/binhvu/workspace/sm-research/.venv01/lib/python3.11/site-packages/pys

In [ ]:
len(lst)

0

In [19]:
lst[3]

{'id': '92656934',
 'ns': '0',
 'title': 'Q93565832',
 'revision': {'id': '1704664990',
  'parent_id': '1627014399',
  'timestamp': '2022-08-15T13:34:26Z',
  'model': 'wikibase-item',
  'format': 'application/json',
  'text': None}}

In [13]:
lst[0]

{'id': '118189804',
 'ns': '146',
 'title': 'Lexeme:L1223080',
 'revision': {'id': '2037511446',
  'parent_id': '2037511365',
  'timestamp': '2023-12-26T14:23:14Z',
  'model': 'wikibase-lexeme',
  'format': 'application/json',
  'text': '{"type":"lexeme","id":"L1223080","lemmas":{"nb":{"language":"nb","value":"s\\u00f8lvplett"}},"lexicalCategory":"Q1084","language":"Q25167","claims":{"P5185":[{"mainsnak":{"snaktype":"value","property":"P5185","hash":"432320b20cb3975ff4969b6ade525ca6bd81ca47","datavalue":{"value":{"entity-type":"item","numeric-id":499327,"id":"Q499327"},"type":"wikibase-entityid"}},"type":"statement","id":"L1223080$BC6DD6C9-4295-4DD3-A53C-8248448813C0","rank":"normal"}],"P10042":[{"mainsnak":{"snaktype":"value","property":"P10042","hash":"f339823c029faa1033082578824cca4960d7e5ae","datavalue":{"value":"59291","type":"string"}},"type":"statement","id":"L1223080$9B925DC0-E535-424D-AF28-7DE24FC14AC8","rank":"normal"}],"P5911":[{"mainsnak":{"snaktype":"value","property":"P59

In [4]:
import bz2, orjson
from lxml import etree
import xml.etree.ElementTree as ET
from io import StringIO, BytesIO
from pathlib import Path
from serde.helper import get_open_fn, get_compression
from typing import Optional

In [5]:
wdcfg = WikidataDirCfg.get_instance()

In [6]:
files = wdcfg.get_page_article_dump_files()

In [7]:
import psutil

process = psutil.Process()

In [8]:
print(process.memory_info().rss / 1024 / 1024)

165.875


In [9]:
def iter_parse_page_article_dump(infile: Path):
    compression = get_compression(infile)
    if compression == "bz2":
        init_decompressor = bz2.BZ2Decompressor
    else:
        raise NotImplementedError(f"Unsupported compression: {compression}")

    parser = etree.XMLPullParser(events=("start", "end"))
    # parser = ET.XMLPullParser(events=("start", "end"))
    decompressor = init_decompressor()

    with get_open_fn(infile)(infile, "r") as f:
        for chunk in iter(lambda: f.read(16384), b""):
            parser.feed(chunk)
            for event, el in parser.read_events():
                yield event, el

    # with open(infile, "rb") as f:
    #     for chunk in iter(lambda: f.read(16384), b""):
    #         decom_chunk = decompressor.decompress(chunk)
    #         parser.feed(decom_chunk)
    #         #     while not decompressor.eof and not decompressor.needs_input:
    #         #         parser.feed(decompressor.decompress(b""))
    #         for event, el in parser.read_events():
    #             yield event, el

    #         # assert decompressor.needs_input
    #         # if you have end of stream
    #         if decompressor.eof:
    #             old_decompressor = decompressor
    #             decompressor = init_decompressor()
    #             parser.feed(decompressor.decompress(old_decompressor.unused_data))
    #             # while not decompressor.needs_input:
    #             #     parser.feed(decompressor.decompress(b""))
    #             for event, el in parser.read_events():
    #                 yield event, el

    #     if decompressor.unused_data != b"":
    #         parser.feed(decompressor.unused_data)
    #         # while not decompressor.needs_input:
    #         #     parser.feed(decompressor.decompress(b""))
    #         for event, el in parser.read_events():
    #             yield event, el


RELEVANT_NAMESPACES = {"0", "120", "146"}


def extract_page(el: ET.Element) -> Optional[dict]:
    (revision,) = el.findall("{*}revision")

    ns = el.find("{*}ns").text
    if ns not in RELEVANT_NAMESPACES:
        return None

    id = el.find("{*}id").text
    title = el.find("{*}title").text
    format = revision.find("{*}format").text
    text = revision.find("{*}text").text
    if format != "application/json":
        print(format, ns, id, title)
        raise NotImplementedError(f"Unsupported format: {format}")

    data = orjson.loads(text)

    return {
        "id": id,
        "ns": ns,
        "title": title,
        "revision": {
            "id": revision.find("{*}id").text,
            "parent_id": (
                tmp.text if (tmp := revision.find("{*}parentid")) is not None else None
            ),
            "timestamp": revision.find("{*}timestamp").text,
            "model": revision.find("{*}model").text,
            "data": data,
        },
    }


def parse_page_article_dump(infile: Path):
    it = iter_parse_page_article_dump(infile)

    # reparsing the header first to ensure the structure is right
    event, el = next(it)
    assert (
        event == "start"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}mediawiki"
    )

    event, el = next(it)
    assert (
        event == "start"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}siteinfo"
    )
    for event, el2 in it:
        if event == "end" and el2 is el:
            break
    assert el.find("{*}sitename").text == "Wikidata"
    el.clear()
    output = []
    current_page = None
    tag_page = "{http://www.mediawiki.org/xml/export-0.10/}page"
    while True:
        event, el = next(it)
        if current_page is None:
            if event == "end":
                assert el.tag == "{http://www.mediawiki.org/xml/export-0.10/}mediawiki"
                break

            # expect a page, at the most upper level beside root
            assert el.tag == tag_page
            current_page = el
            continue

        if event == "end" and el.tag == tag_page:
            assert current_page is el
            # now we can process current_page
            clean_page = extract_page(current_page)
            if clean_page is not None:
                yield clean_page
                # output.append(clean_page)
                output.append(process.memory_info().rss / 1024 / 1024)
            current_page = None
            el.clear()

    # make sure we end with the root page
    try:
        next(it)
        assert False
    except StopIteration:
        pass

    print("max memory usage", max(output))

In [10]:
count = 0
it = parse_page_article_dump(files[0])
for x in it:
    count += len(x)
count

max memory usage 183.50390625


281448

In [7]:
count = 0
it = parse_page_article_dump(files[0])
for x in it:
    count += len(x)
count

1483.1796875


281448

In [12]:
import gc

gc.collect()

0

In [9]:
print("new memory", process.memory_info().rss / 1024 / 1024)

new memory 178.1875


In [13]:
print(process.memory_info().rss / 1024 / 1024)

1316.3203125


In [11]:
print("default", process.memory_info().rss / 1024 / 1024)

default 6112.01171875


In [7]:
len(el)

NameError: name 'el' is not defined

In [25]:
tree = etree.fromstring(content)

In [53]:
events2 = list(iter_parse_page_article_dump(files[0]))

In [64]:
def parse_page_article_dump2(it):
    # reparsing the header first to ensure the structure is right
    event, el = next(it)
    assert (
        event == "start"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}mediawiki"
    )

    event, el = next(it)
    assert (
        event == "start"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}siteinfo"
    )
    for event, el2 in it:
        if event == "end" and el2 is el:
            break
    assert el.find("{*}sitename").text == "Wikidata"
    output = []
    current_page = None
    tag_page = "{http://www.mediawiki.org/xml/export-0.10/}page"
    for event, el in it:
        last_event, last_el = event, el
        # assert not (
        #     event == "end"
        #     and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}mediawiki"
        # ), (event, el.tag)
        if current_page is None:
            # expect a page, at the most upper level beside root
            assert event == "start" and el.tag == tag_page
            current_page = el
            continue

        if event == "end" and el.tag == tag_page:
            assert current_page is el, (
                current_page,
                el,
                # current_page.find("{*}id").text,
                # el.find("{*}id").text,
            )
            # now we can process current_page
            # clean_page = extract_page(current_page)
            clean_page = None
            if clean_page is not None:
                output.append(clean_page)
            current_page = None

    print(last_event, last_el)
    # make sure we end with the root page
    if not (
        event == "end"
        and el.tag == "{http://www.mediawiki.org/xml/export-0.10/}mediawiki"
    ):
        print("error -- stop before reaching the end", event, el.tag)
        print(etree.tostring(el))
    return output


parse_page_article_dump2(iter(events2))

AssertionError: 

In [59]:
x = ET.iterparse(BytesIO(content))
for a, b in x:
    print(a, b)
    break

end <Element '{http://www.mediawiki.org/xml/export-0.10/}sitename' at 0x7f9248f81c60>


In [48]:
events = list(ET.iterparse(BytesIO(content)))

In [60]:
[(k, e.tag) for k, e in events2[-10:]]

[('end', '{http://www.mediawiki.org/xml/export-0.10/}model'),
 ('start', '{http://www.mediawiki.org/xml/export-0.10/}format'),
 ('end', '{http://www.mediawiki.org/xml/export-0.10/}format'),
 ('start', '{http://www.mediawiki.org/xml/export-0.10/}text'),
 ('end', '{http://www.mediawiki.org/xml/export-0.10/}text'),
 ('start', '{http://www.mediawiki.org/xml/export-0.10/}sha1'),
 ('end', '{http://www.mediawiki.org/xml/export-0.10/}sha1'),
 ('end', '{http://www.mediawiki.org/xml/export-0.10/}revision'),
 ('end', '{http://www.mediawiki.org/xml/export-0.10/}page'),
 ('end', '{http://www.mediawiki.org/xml/export-0.10/}mediawiki')]

In [54]:
len(events2)

2257070

In [52]:
len(events)

1128535

In [37]:
print(content[-50000:].decode())

ot;value&quot;:&quot;\u062f \u0648\u064a\u06a9\u064a\u0645\u06d0\u0689\u064a\u0627 \u0648\u06d0\u0634\u0646\u064a\u0632\u0647&quot;},&quot;sd&quot;:{&quot;language&quot;:&quot;sd&quot;,&quot;value&quot;:&quot;\u0632\u0645\u0631\u0648:\u0648\u06aa\u064a\u067e\u064a\u068a\u064a\u0627 \u0632\u0645\u0631\u0627 \u0628\u0646\u062f\u064a&quot;},&quot;pam&quot;:{&quot;language&quot;:&quot;pam&quot;,&quot;value&quot;:&quot;Kategoriya ning Wikimedia&quot;},&quot;arz&quot;:{&quot;language&quot;:&quot;arz&quot;,&quot;value&quot;:&quot;\u062a\u0635\u0646\u064a\u0641 \u0628\u062a\u0627\u0639 \u0648\u064a\u0643\u064a\u0645\u064a\u062f\u064a\u0627&quot;},&quot;kw&quot;:{&quot;language&quot;:&quot;kw&quot;,&quot;value&quot;:&quot;Wikimedia:Klassys&quot;},&quot;ia&quot;:{&quot;language&quot;:&quot;ia&quot;,&quot;value&quot;:&quot;categoria Wikimedia&quot;},&quot;ur&quot;:{&quot;language&quot;:&quot;ur&quot;,&quot;value&quot;:&quot;\u0648\u06cc\u06a9\u06cc\u0645\u06cc\u0688\u06cc\u0627 \u0632\u0645\u0631

In [248]:
files[0]

PosixPath('/nas/home/binhvu/kgdata/wikidata/20240220/000_dumps/pages-articles/wikidatawiki-20240220-pages-articles-multistream6.xml-p5969005p6052571.bz2')

In [249]:
content[-100:]

b'}}}</text>\n      <sha1>fp3dzoi53boabe210d2v40rc58t6474</sha1>\n    </revision>\n  </page>\n</mediawiki>'

In [216]:
list(el.iter())

[<Element '{http://www.mediawiki.org/xml/export-0.10/}page' at 0x7fc87faf24d0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}title' at 0x7fc81c8034c0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}ns' at 0x7fc81c800590>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}id' at 0x7fc81c800a90>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}revision' at 0x7fc81c4e0e50>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}id' at 0x7fc81c4e0270>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}parentid' at 0x7fc81c4e03b0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}timestamp' at 0x7fc81c4e1c10>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}contributor' at 0x7fc81c4e0860>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}username' at 0x7fc81c4e0040>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}id' at 0x7fc81c4e0cc0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}comment' at 0x7fc81c4e18f0>,
 <Element '{http://w

In [177]:
el

<Element '{http://www.mediawiki.org/xml/export-0.10/}siteinfo' at 0x7fc81c8f1620>

[<Element '{http://www.mediawiki.org/xml/export-0.10/}revision' at 0x7fc81c4e0e50>]

In [221]:
el.find("{*}revision/{*}id").text

'1997397285'

In [180]:
el.findall("{http://www.mediawiki.org/xml/export-0.10/}namespace")

[]

In [178]:
list(el.iter())

[<Element '{http://www.mediawiki.org/xml/export-0.10/}siteinfo' at 0x7fc81c8f1620>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}sitename' at 0x7fc81c96b8d0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}dbname' at 0x7fc81c969b70>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}base' at 0x7fc81c97f1a0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}generator' at 0x7fc81c97c8b0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}case' at 0x7fc81c7a8d60>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}namespaces' at 0x7fc81c7ab150>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}namespace' at 0x7fc81c7a8f40>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}namespace' at 0x7fc81c6fd8a0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}namespace' at 0x7fc81c6fcea0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}namespace' at 0x7fc81c6fdbc0>,
 <Element '{http://www.mediawiki.org/xml/export-0.10/}namespace' at 0x7fc81c6fc

In [21]:
type(content)

bytes

In [10]:
f = BytesIO(content)

In [11]:
it = ET.iterparse(f, events=("start", "end"))

In [145]:
event, el = next(it)
event, el

('start',
 <Element '{http://www.mediawiki.org/xml/export-0.10/}text' at 0x7fc81c8f1490>)

In [153]:
el

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'name'

In [147]:
import orjson

orjson.loads(el.text)

{'type': 'item',
 'id': 'Q6173088',
 'labels': {'en': {'language': 'en', 'value': 'Jeevandran Nair'},
  'ta': {'language': 'ta', 'value': 'ஜீவேந்திரன் நாயர்'},
  'fr': {'language': 'fr', 'value': 'Jeevandran Nair'},
  'nl': {'language': 'nl', 'value': 'Jeevandran Nair'},
  'ast': {'language': 'ast', 'value': 'Jeevandran Nair'},
  'es': {'language': 'es', 'value': 'Jeevandran Nair'},
  'sq': {'language': 'sq', 'value': 'Jeevandran Nair'},
  'ga': {'language': 'ga', 'value': 'Jeevandran Nair'},
  'arz': {'language': 'arz', 'value': 'چيڤاندران ناير'}},
 'descriptions': {'en': {'language': 'en', 'value': 'Malaysian cricketer'},
  'fa': {'language': 'fa', 'value': 'بازیکن کریکت اهل مالزی'},
  'nl': {'language': 'nl', 'value': 'cricketspeler uit Maleisië'},
  'ar': {'language': 'ar', 'value': 'لاعب كركيت ماليزي'},
  'bn': {'language': 'bn', 'value': 'মালয়েশীয় ক্রিকেটার'},
  'ast': {'language': 'ast', 'value': 'xugador de críquet malasiu'},
  'ga': {'language': 'ga', 'value': 'cruicéadaí Ma

In [24]:
tree = etree.fromstring(content.decode())

In [27]:
tree

['__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_init',
 'addnext',
 'addprevious',
 'append',
 'attrib',
 'base',
 'clear',
 'cssselect',
 'extend',
 'find',
 'findall',
 'findtext',
 'get',
 'getchildren',
 'getiterator',
 'getnext',
 'getparent',
 'getprevious',
 'getroottree',
 'index',
 'insert',
 'items',
 'iter',
 'iterancestors',
 'iterchildren',
 'iterdescendants',
 'iterfind',
 'itersiblings',
 'itertext',
 'keys',
 'makeelement',
 'nsmap',
 'prefix',
 'remove',
 'replace',
 'set',
 'sourceline',
 'tag',
 'tail',
 'text',
 'values',
 'xp

In [13]:
!du -hs {files[0]}

99M	/nas/home/binhvu/kgdata/wikidata/20240220/000_dumps/pages-articles/wikidatawiki-20240220-pages-articles-multistream6.xml-p5969005p6052571.bz2


In [182]:
print(content[:25000].decode())

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en">
  <siteinfo>
    <sitename>Wikidata</sitename>
    <dbname>wikidatawiki</dbname>
    <base>https://www.wikidata.org/wiki/Wikidata:Main_Page</base>
    <generator>MediaWiki 1.42.0-wmf.18</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Media</namespace>
      <namespace key="-1" case="first-letter">Special</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Talk</namespace>
      <namespace key="2" case="first-letter">User</namespace>
      <namespace key="3" case="first-letter">User talk</namespace>
      <namespace key="4" case="first-letter">Wikidata</namespace>
      <namespace key="5" case="first-letter">Wikidata talk</namespa

In [10]:
len(content) / 1024 / 1024 / 1024

1.505859324708581

#### Test multi-lingual string compression

In [3]:
ds = import_dataset('wikidata.entity_metadata')

In [4]:
ds.get_rdd().count()

2023-08-25 08:46:07.421 | DEBUG    | kgdata.spark:get_spark_context:90 - Spark Options: [('spark.master', 'spark://ckg04.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g'), ('spark.driver.maxResultSize', '128g')]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/25 08:46:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


103326088

In [9]:
112858294 - 103326088

9532206

In [4]:
ds.get_rdd().take(1)

[{'id': '5969005',
  'ns': '0',
  'title': 'Q6173088',
  'revision': {'id': '1997397285',
   'parent_id': '1495648365',
   'timestamp': '2023-10-26T03:00:36Z',
   'model': 'wikibase-item',
   'format': 'application/json',
   'text': '{"type":"item","id":"Q6173088","labels":{"en":{"language":"en","value":"Jeevandran Nair"},"ta":{"language":"ta","value":"\\u0b9c\\u0bc0\\u0bb5\\u0bc7\\u0ba8\\u0bcd\\u0ba4\\u0bbf\\u0bb0\\u0ba9\\u0bcd \\u0ba8\\u0bbe\\u0baf\\u0bb0\\u0bcd"},"fr":{"language":"fr","value":"Jeevandran Nair"},"nl":{"language":"nl","value":"Jeevandran Nair"},"ast":{"language":"ast","value":"Jeevandran Nair"},"es":{"language":"es","value":"Jeevandran Nair"},"sq":{"language":"sq","value":"Jeevandran Nair"},"ga":{"language":"ga","value":"Jeevandran Nair"},"arz":{"language":"arz","value":"\\u0686\\u064a\\u06a4\\u0627\\u0646\\u062f\\u0631\\u0627\\u0646 \\u0646\\u0627\\u064a\\u0631"}},"descriptions":{"en":{"language":"en","value":"Malaysian cricketer"},"fa":{"language":"fa","value":"\\u0

In [5]:
us = ds.get_rdd().filter(lambda x: x["title"] == "Q30").collect()

In [11]:
ds.get_rdd().map(lambda x: x["title"]).distinct().saveAsTextFile(
    "/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/050_abc"
)

['{"id":"7552572","ns":"0","title":"Q7631747","revision":{"id":"1067190908","parent_id":"935669316","timestamp":"2019-12-01T12:19:28Z","model":"wikibase-item","format":"application/json","text":"{\\"type\\":\\"item\\",\\"id\\":\\"Q7631747\\",\\"labels\\":{\\"en\\":{\\"language\\":\\"en\\",\\"value\\":\\"Subodh College\\"}},\\"descriptions\\":[],\\"aliases\\":[],\\"claims\\":{\\"P856\\":[{\\"mainsnak\\":{\\"snaktype\\":\\"value\\",\\"property\\":\\"P856\\",\\"hash\\":\\"6c5fc0e1523302d70647ce38caf870ed9da21456\\",\\"datavalue\\":{\\"value\\":\\"http:\\\\/\\\\/www.subodhpgcollege.com\\",\\"type\\":\\"string\\"}},\\"type\\":\\"statement\\",\\"id\\":\\"Q7631747$28B2ACD9-B13D-40C3-AF52-92F6E62B2847\\",\\"rank\\":\\"normal\\",\\"references\\":[{\\"hash\\":\\"fa278ebfc458360e5aed63d5058cca83c46134f1\\",\\"snaks\\":{\\"P143\\":[{\\"snaktype\\":\\"value\\",\\"property\\":\\"P143\\",\\"hash\\":\\"e4f6d9441d0600513c4533c672b5ab472dc73694\\",\\"datavalue\\":{\\"value\\":{\\"entity-type\\":\\"item\

In [12]:



get_spark_context().textFile(
    "/nas/ckgfs/users/binhvu/kgdata-2/wikidata/20240220/050_abc"
).count()

112858294

In [10]:
us

[{'id': '126',
  'ns': '0',
  'title': 'Q30',
  'revision': {'id': '2082071098',
   'parent_id': '2081790135',
   'timestamp': '2024-02-20T04:58:51Z',
   'model': 'wikibase-item',
   'format': 'application/json',
   'text': '{"type":"item","id":"Q30","labels":{"el":{"language":"el","value":"\\u0397\\u03bd\\u03c9\\u03bc\\u03ad\\u03bd\\u03b5\\u03c2 \\u03a0\\u03bf\\u03bb\\u03b9\\u03c4\\u03b5\\u03af\\u03b5\\u03c2 \\u0391\\u03bc\\u03b5\\u03c1\\u03b9\\u03ba\\u03ae\\u03c2"},"ay":{"language":"ay","value":"Istadus Unidus"},"pnb":{"language":"pnb","value":"\\u0627\\u0645\\u0631\\u06cc\\u06a9\\u0627"},"na":{"language":"na","value":"Eben Merika"},"mk":{"language":"mk","value":"\\u0421\\u0410\\u0414"},"zu":{"language":"zu","value":"IMelika"},"bpy":{"language":"bpy","value":"\\u09a4\\u09bf\\u09b2\\u09aa\\u09be\\u09b0\\u09be\\u09b7\\u09cd\\u099f\\u09cd\\u09b0"},"lt":{"language":"lt","value":"Jungtin\\u0117s Amerikos Valstijos"},"jam":{"language":"jam","value":"Yunaitid Stiet"},"sk":{"language":"sk","

In [7]:
us[0]

{'id': '126',
 'ns': '0',
 'title': 'Q30',
 'revision': {'id': '2082071098',
  'parent_id': '2081790135',
  'timestamp': '2024-02-20T04:58:51Z',
  'model': 'wikibase-item',
  'format': 'application/json',
  'text': '{"type":"item","id":"Q30","labels":{"el":{"language":"el","value":"\\u0397\\u03bd\\u03c9\\u03bc\\u03ad\\u03bd\\u03b5\\u03c2 \\u03a0\\u03bf\\u03bb\\u03b9\\u03c4\\u03b5\\u03af\\u03b5\\u03c2 \\u0391\\u03bc\\u03b5\\u03c1\\u03b9\\u03ba\\u03ae\\u03c2"},"ay":{"language":"ay","value":"Istadus Unidus"},"pnb":{"language":"pnb","value":"\\u0627\\u0645\\u0631\\u06cc\\u06a9\\u0627"},"na":{"language":"na","value":"Eben Merika"},"mk":{"language":"mk","value":"\\u0421\\u0410\\u0414"},"zu":{"language":"zu","value":"IMelika"},"bpy":{"language":"bpy","value":"\\u09a4\\u09bf\\u09b2\\u09aa\\u09be\\u09b0\\u09be\\u09b7\\u09cd\\u099f\\u09cd\\u09b0"},"lt":{"language":"lt","value":"Jungtin\\u0117s Amerikos Valstijos"},"jam":{"language":"jam","value":"Yunaitid Stiet"},"sk":{"language":"sk","value":"S

In [7]:
def compress_multilingual_string(s):
    values = []
    existing_values = {}
    lang2index = {}
    for lang, value in s.lang2value.items():
        if value not in existing_values:
            values.append(value)
            existing_values[value] = len(values) - 1
        lang2index[lang] = existing_values[value]
    
    return [values, lang2index]
    return {
        "values": values,
        "lang2index": lang2index,
        "lang": s.lang
    }

In [30]:
import orjson

def measure(label):
    size = len(orjson.dumps(label.to_dict()))
    size2 = len(orjson.dumps(compress_multilingual_string(label)))
    
    return size, size2
    
res = ds.get_rdd().map(lambda e: e.label).map(measure).fold((0, 0), lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1]))
print(M.percentage(res[1], res[0], M.datasize))

62.92% (17.0GiB/27.0GiB)


----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 48154)
Traceback (most recent call last):
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/data/binhvu/gramsplus/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/data/binhvu/gramsplus/.venv/lib/python3.11/site-packages/pyspark/accumul

In [29]:
res = ds.get_rdd().map(lambda e: e.description).map(measure).fold((0, 0), lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1]))
print(M.percentage(res[1], res[0], M.datasize))

95.05% (95.7GiB/100.7GiB)
